In [ ]:
!pip install langchain openai

In [ ]:
!pip install googletrans==4.0.0-rc1
from googletrans import Translator


In [ ]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
import os

In [ ]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.2 which is incompatible.


In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap
import langchain_google_genai
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
conversation_history=[]
def add_to_convo(user_input,bot_response):
  conversation_history.append(f"User:{user_input}")
  conversation_history.append(f"Bot:{bot_response}")

In [ ]:
import pandas as pd

def save():
  df = pd.DataFrame(conversation_history, columns=['Conversation']) # Changed to one column to match data shape
  df.to_csv('conversation_history.csv', index=False)

In [ ]:
def sentiment_analyzer(text):
  prompt="analyse the text of this complaint or feedback given,just give the tone and other observations"
  message=HumanMessage(content=f"{prompt}")
  response=llm.invoke([message])



In [ ]:
translator=Translator()
def chatbot(input_text,output_language):
    detected_language = translator.detect(input_text).lang
    translated_input = translator.translate(input_text, dest='en').text
    message = HumanMessage(
        content=f"you are a chatbot for an e commerce website equivalent to amazon or flipkart. the user is giving you {input_text}...derive the sentiments from sentiment_analyzer({input_text}) and generate a good respone or a humanly respone so as to mantian the interest...prioritize accordingly to the sentiments. ask for any further information required by the user and save the previous information. do not print the sentiment analysis,just print the response....and take inspiration from order tracking of amazon. also consider a feature that a customer can change his delivery time or payment method using chatbot but verify the customer first" # Call sentiment_analyzer function with input_text
    )
    response = llm.invoke([message])
    translated_response=response.content
    translated_response = translator.translate(translated_response, dest=output_language).text
    print(f"Chatbot: {translated_response}")
    add_to_convo(input_text,response.content)
    translated_response=response
    save()
chatbot("मुझे प्राप्त ऑर्डर पूरी तरह से खराब गुणवत्ता का है, मैं इस सेवा से बहुत निराश हूं और अपना ऑर्डर वापस करना चाहता हूं, इसके लिए चरण बताएं","spanish")

Chatbot: Lamento saber que recibió un pedido de mala calidad.Entiendo su decepción y me gustaría ayudarlo a devolver el pedido.

Para devolver un pedido, siga estos pasos:

1. Vaya a la sección "Mis órdenes" de su cuenta.
2. Encuentre el pedido que desea devolver y haga clic en "Elemento de retorno".
3. Seleccione el motivo de su devolución y proporcione cualquier información adicional que se requiera.
4. Haga clic en "Enviar solicitud de devolución".

Una vez que haya enviado su solicitud de devolución, la revisaremos y le enviaremos un correo electrónico de confirmación con más instrucciones.

Si tiene alguna otra pregunta o requiere más ayuda, no dude en preguntar.

**Información adicional:**

* También puede cambiar su tiempo de entrega o método de pago siguiendo estos pasos:
* Vaya a la sección "Mis órdenes" de su cuenta.
* Encuentre el pedido que desea modificar y haga clic en "Cambiar el tiempo de entrega" o "Cambiar el método de pago".
* Seleccione la opción deseada y haga clic

In [ ]:
import pandas as pd

def summarize(adress):
  conversation_data=pd.read_csv(f"{adress}")
  conversation_text = ""
  for index, row in conversation_data.iterrows():
        # Access the conversation from the 'Conversation' column and split into user and bot parts
        parts = row['Conversation'].split(':')
        if len(parts) >= 2:  # Make sure there's at least a user and bot part
          user_part = parts[0]
          bot_part = parts[1]
          conversation_text += f"{user_part}: {bot_part} \n\n"
  prompt = f"""
  The issue that the customer had couldn't be solved using the chatbot,
  and now the customer service agent has to come into play.
  Summarize the chats done by the customer and chatbot. Here is the conversation:
  {conversation_text}
  """

  message = HumanMessage(content=prompt)
  response = llm.invoke([message])
  print(response.content)

summary = summarize('/content/conversation_history.csv')
print(summary)


The customer has been consistently reporting dissatisfaction with the quality of an order. Despite the chatbot providing instructions for returning the order five times, the customer has not acknowledged or followed the instructions. The customer has repeated their request for return instructions in multiple messages.
None
